In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from IPython.display import display, Markdown, Latex
import itertools

def lorentzian(f, I, f_0, S, a, b):
    return I/(1+((f-f_0)/S)**2)+a*f+b     #Lorentziana con correzione lineare

def frequenza(T,A,B,C,D,E,TC):
    return A*(1+(B*C/(D*(1-(T/TC)**2)**(1/2)))*(1/(np.tanh((E/C)*(1-(T/TC)**2)**(1/2)))))**(-1/2)

prova_1 = pd.read_csv("Car_Prova_1.txt")   #file txt da caricare manualmente
prova_2 = pd.read_csv("Car_Prova_2.txt")
prova_3 = pd.read_csv("Car_Prova_3.txt")
prova_4 = pd.read_csv("Car_Prova_4.txt")
prova_5 = pd.read_csv("Car_Prova_5.txt")
prova_6 = pd.read_csv("Salto_Def.txt")

guess_1 = [23.5, 115, 10, 0.01, 0.15]  #guess da inserire manualmente
guess_2 = [23.5, 115, 10, 0.01, 0.15]
guess_3 = [23.5, 124, 10, 0.01, 0.15]
guess_4 = [0.8, 1, -0.08, 0.01, 0.15]
guess_5 = [0.8, 1, -0.08, 0.01, 0.15]
guess_6 = [126.34,1.91,1.63,21816,0.037,90.1] 

temp = prova_6["Temperatura potenziometro"].values
freq = prova_6["Frequenze"].values
x = [prova_6["Temperatura potenziometro"][i] for i in range(7, len(freq))]
z = [prova_6["Frequenze"][i] for i in range(7, len(temp))]

guesses = [guess_1, guess_2, guess_3, guess_4, guess_5]
prove = [prova_1, prova_2, prova_3, prova_4, prova_5]
prove1 = [prova_1, prova_2, prova_3]

freqs, guads = [x["Frequenze"].values for x in prove1], [x['Guadagni'].values for x in prove1]

p_1, cov_1 = curve_fit(lorentzian, freqs[0], guads[0], guesses[0])   #fit di ogni coppia di valori frequenza/guadagno
p_2, cov_2 = curve_fit(lorentzian, freqs[1], guads[1], guesses[1])
p_3, cov_3 = curve_fit(lorentzian, freqs[2], guads[2], guesses[2])
p_4, cov_4 = curve_fit(lorentzian, prova_4["Frequenze normalizzate"], prova_4["Guadagni normalizzati"], guesses[3])
p_5, cov_5 = curve_fit(lorentzian, prova_5["Frequenze normalizzate"], prova_5["Guadagni normalizzati"], guesses[4])
p_6, cov_6 = curve_fit(frequenza, z, x, guess_6)

ps, covs = [p_1, p_2, p_3, p_4, p_5], [cov_1, cov_2, cov_3, cov_4, cov_5]     #set di parametri del fit

f_ris_1=p_1[1]       #frequenze di risonanza
f_ris_2=p_2[1]
f_ris_3=p_3[1]
f_ris_4=p_4[1]*94.0
f_ris_5=p_5[1]*66.3

y_1, y_2, y_3, y_4, y_5 = [], [], [], [], []        #liste inizializzate vuote che saranno riempite con i valori di fit dell'asse y

Set_Color_1 = ["r", "m", "c", "coral", "g"]
Set_Color_2 = ["black", "g", "b", "gray", "r"] 

ys = [y_1, y_2, y_3, y_4, y_5]

f_risonanze = [f_ris_1, f_ris_2, f_ris_3, f_ris_4, f_ris_5]

for m,k,h in zip(f_risonanze, ps, range(1,len(ps)+1)):
    tuplex=(h,)+tuple(k)
    display(Latex('Fitting parameters test %5.0f): $\quad$ I = %5.2f, $\quad$ $\omega_0$ = %5.3f $\quad$ S = %5.2f, $\quad$ a = %5.2f, $\quad$ b = %5.2f' %tuplex))
    display(Latex('The resonance frequency is $\simeq$ %5.1f Hz\n' %m))
    #print("\n")       #fitting parameters test

for k,l,m,f in zip(ys, prove1, ps, freqs):
    for j in range(len(f)):
        k.append(lorentzian(l["Frequenze"][j], m[0], m[1], m[2], m[3], m[4])) #riempie le liste vuote y tramite fit con lorentziana e parametri       

for j in range(len(prova_4["Frequenze normalizzate"].values)):
        y_4.append(lorentzian(prova_4["Frequenze normalizzate"][j], p_4[0], p_4[1], p_4[2], p_4[3], p_4[4])) 
        
for j in range(len(prova_5["Frequenze normalizzate"].values)):
        y_5.append(lorentzian(prova_5["Frequenze normalizzate"][j], p_5[0], p_5[1], p_5[2], p_5[3], p_5[4])) 
        
y_6=[]

for j in range(7,len(freq)):
        y_6.append(frequenza(prova_6["Temperatura potenziometro"][j], p_6[0], p_6[1], p_6[2], p_6[3], p_6[4],p_6[5]))        
             
points, fit = "g","b"

fig, axes = plt.subplots(3, 2, squeeze = False)    #crea array 2D per rappresentare i grafici
plt.subplots_adjust(left=-1.5, bottom=None, right=1, top=4.5, wspace=0.2, hspace=0.15)  #regola la posizione dei grafici
  
indici = [(0,0), (0,1), (1,0)] #crea gli indici dell'array di figure

for k,y,m,h,t,a in zip(prove1, ys, ps, indici, Set_Color_1, Set_Color_2):
    axes[h[0]][h[1]].plot(k['Frequenze'], k['Guadagni'], "o", color = a, label = 'Exp. data')
    axes[h[0]][h[1]].plot(k['Frequenze'], y, t, label = '\nFit parameters:\n\n I = %5.3f\n $\omega_0$ = %5.3f\n S = %5.3f\n a = %5.3f\n b = %5.3f' % tuple(m))
    axes[h[0]][h[1]].set(xlabel = '$f$  [kHz]', ylabel = 'Gain')
    axes[h[0]][h[1]].legend(loc="upper right")
    
axes[1][1].plot(prova_4["Frequenze normalizzate"], prova_4["Guadagni normalizzati"], "o", color = Set_Color_2[3], label = 'Exp. data 4.7 nF')
axes[1][1].plot(prova_5["Frequenze normalizzate"], prova_5["Guadagni normalizzati"], "o", color = Set_Color_2[4], label = 'Exp. data 10 nF')
axes[1][1].plot(prova_4['Frequenze normalizzate'], y_4, Set_Color_1[3], label = '\nFit parameters 4.7 nF:\n\n I = %5.3f\n $\omega_0$ = %5.3f\n S = %5.3f\n a = %5.3f\n b = %5.3f' % tuple(p_4))
axes[1][1].plot(prova_5['Frequenze normalizzate'], y_5, Set_Color_1[4], label = '\nFit parameters 10 nF:\n\n I = %5.3f\n $\omega_0$ = %5.3f\n S = %5.3f\n a = %5.3f\n b = %5.3f' % tuple(p_5))
axes[1][1].set(xlabel = '$\omega/\omega_0$', ylabel = 'Normalized Gain')
axes[1][1].legend(loc="upper right")

axes[2][0].plot(prova_6["Temperatura potenziometro"], prova_6["Frequenze"], "bo", color = points, label = 'Exp. data')
axes[2][0].plot(x, y_6, fit, label = 'theoretical model')
axes[2][0].set(xlabel = '$T$  [°K]', ylabel = '$f_r$  [kHz]')
axes[2][0].legend()

fig.delaxes(axes[2][1])

<ipython-input-2-01b3aeb2871c>:13: RuntimeWarning: invalid value encountered in sqrt
  return A*(1+(B*C/(D*(1-(T/TC)**2)**(1/2)))*(1/(np.tanh((E/C)*(1-(T/TC)**2)**(1/2)))))**(-1/2)


RuntimeError: Optimal parameters not found: Number of calls to function has reached maxfev = 1400.